Steps:
- Scarping Job postings with Selenium and bs4
- Visualize in Tableau or PowerBI

import necessary modules - first try with BeautifulSoup

In [97]:
import driver as driver
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
import datetime
start_time = time()

from warnings import warn


Select wanted url

In [98]:
job_kind = str(input('Please enter the kind of job you want to work in (e.g. Data Analyst)'))
city = str(input('Please enter the city you want to work in'))
state = str(input('Please enter the state you want to work in (not mandatory)'))
country = str(input('Please enter the country you want to work in'))
url = 'https://www.linkedin.com/jobs/search?keywords=' + job_kind + '&location='+ city + '%2C%20'+ state +'%2C%20'+ country

no_of_jobs = 51

url



'https://www.linkedin.com/jobs/search?keywords=Consulting&location=Paris%2C%20%2C%20France'

open windows with url stated above in

In [99]:
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

to show more jobs than the initial 25

In [100]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(20)



parsing the visible webpage

In [101]:
pageSource = driver.page_source
soup = BeautifulSoup(pageSource,  "html.parser")



Parsing content

In [102]:
first_job = soup.find('h3', class_ = 'result-card__title job-result-card__title').text
print(first_job)

first_company = soup.find('a', class_ = 'result-card__subtitle-link job-result-card__subtitle-link').text
print(first_company)

Consultant
HYFFN


Create job container

In [103]:
job_container = soup.find('ul', class_="jobs-search__results-list")
print(len(job_container))

42


create for loop to look for all jobs

In [104]:
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 42 jobs.


In [105]:
job_titles = []
companies = []
locations = []
time = []
date = []


for jobs in job_container:
    post_title = jobs.find('h3', class_ = 'result-card__title job-result-card__title').text
    job_titles.append(post_title)

    post_company = jobs.find('h4', class_ = 'result-card__subtitle job-result-card__subtitle').text
    companies.append(post_company)

    post_locations = jobs.find('span', class_ = 'job-result-card__location').text
    locations.append(post_locations)

    post_date = jobs.select_one('time')['datetime']
    date.append(post_date)


In [106]:
ml = {'Title': job_titles, 'Company': companies, 'Locations': locations, 'Post Date': date}
print(ml)

{'Title': ['Consultant', 'Consultant', 'Consultant.e Junior - Accenture Consulting - CDI', 'Business Consulting Associate Consultant Amsterdam', 'Project Assistant', 'Responsable du développement -Fintech H/F - FR8246687', 'Social Impact PA Intern', 'Analyst / Junior Associate (M/F)', 'Analyst - Oslo office', 'Implementation Consultant', 'Consultant - Capital Excellence', 'Analyst (Paris/Brussels)', 'Consultant PMO junior Transformation digitale – Paris', 'Consultant junior', 'Mergers and Acquisitions Associate', 'Associate Partner', 'Manager, Client Consulting - Visa Consulting & Analytics', 'Marketing Asssistant Intern', 'Solutions Consultant - AdobeSign', 'Consultant junior en Innovation Paris H/F', 'Business Transformation Consultant', 'Buyout Internship', 'Intern', 'Head of Impact & Sustainability Consulting', 'Consultant', 'Business Consulting Manager', 'Senior Consultant / Associate, Strategic Consulting', 'Consultant Junior en Stratégie Data et IA', 'Intern - Data and Analytics

In [107]:
df = pd.DataFrame(ml)
df.head(1)

,Title,Company,Locations,Post Date
0,Consultant,HYFFN,"Paris, Île-de-France, France",2021-02-03


Separate City from State and Country


In [108]:
new = df.Locations.str.split(',', n = 0, expand = True)

In [109]:
new2 = new.rename(columns = {0: 'City2'})

In [110]:
new2


,City2,1,2
0,Paris,Île-de-France,France
1,Paris,Île-de-France,France
2,Paris,Île-de-France,France
3,Paris,Île-de-France,France
4,Paris,Île-de-France,France
5,Paris,Île-de-France,France
6,Paris,Île-de-France,France
7,Paris,Île-de-France,France
8,Paris,Île-de-France,France
9,Paris,Île-de-France,France


merge df and new, drop Locations as it is redundant now

In [111]:
df2 = df.join(new2, how = 'left')
df2.drop(columns = ['Locations',1 , 2], inplace = True)
df2.head(4)

,Title,Company,Post Date,City2
0,Consultant,HYFFN,2021-02-03,Paris
1,Consultant,pnlconseil,2021-02-03,Paris
2,Consultant.e Junior - Accenture Consulting - CDI,Accenture France,2021-01-29,Paris
3,Business Consulting Associate Consultant Amste...,Synechron,2021-01-25,Paris


In [112]:
new3 = df2.City2.str.split(' ', expand=True)
new4 = new3.rename(columns = {0: 'City'})
new5 = new4.City
new5.head()

0    Paris
1    Paris
2    Paris
3    Paris
4    Paris
Name: City, dtype: object

In [113]:
df4 = df2.join(new5, how = 'left')
df4.drop(columns = ["City2"], inplace = True)
df4.set_index('Title', inplace = True)
df4

,Company,Post Date,City
Title,,,
Consultant,HYFFN,2021-02-03,Paris
Consultant,pnlconseil,2021-02-03,Paris
Consultant.e Junior - Accenture Consulting - CDI,Accenture France,2021-01-29,Paris
Business Consulting Associate Consultant Amsterdam,Synechron,2021-01-25,Paris
Project Assistant,"UNESCO - United Nations Educational, Scientifi...",2021-02-02,Paris
Responsable du développement -Fintech H/F - FR8246687,Morgan Philips Group,2021-02-03,Paris
Social Impact PA Intern,Boston Consulting Group (BCG),2021-01-21,Paris
Analyst / Junior Associate (M/F),Ergon Capital,2021-02-03,Paris
Analyst - Oslo office,"Bryan, Garnier & Co",2021-01-28,Paris


create CSV, with todays Date

In [114]:
date_object = datetime.date.today()
date = str(date_object) + '_'
date

df4.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/' + date + city + '_' + country + '.csv')

concat new jobs to existing list

In [115]:
final_df = pd.read_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')
final_df.set_index('Title', inplace = True)
final_df.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


In [116]:
dfWIP = pd.concat([final_df, df4], axis=0)
dfWIP.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


drop duplicated rows

In [117]:
new_final_df = dfWIP.drop_duplicates()
new_final_df

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt
...,...,...,...
Consultant,Oresys,2020-12-28,Paris
Consultant,fab group,2021-01-21,Paris
Consultant interne La Banque Postale Consulting H/F,La Banque Postale,2021-01-15,Paris


In [118]:
new_final_df.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')

